In [1]:
%jsroot on

In [2]:
#include <TFile.h>
#include <iostream>
#include <string>
#include <stdio.h>

In [3]:
TFile *f=new TFile("/home/student/HZZ4l/Analysis/Output_HZZAnalysis/data.root");

In [4]:
mass_four_lep->Print();

TH1.Print Name  = mass_four_lep, Entries= 321, Total sum= 33


In [5]:
TH1F *m_four_lep=(TH1F*)f->Get("mass_four_lep");

In [6]:
TCanvas *cz = new TCanvas("cz","cz",10,10,900,600);
m_four_lep->Draw();
cz->Draw();

In [7]:
TF1 *f1= new TF1("f1","gaus",115,130);
m_four_lep->Fit("f1");
m_four_lep->Draw();
cz->Draw();

 FCN=5.82062 FROM MIGRAD    STATUS=CONVERGED      81 CALLS          82 TOTAL
                     EDM=3.74175e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.29854e+00   6.32843e-01   5.61863e-04  -1.16888e-03
   2  Mean         1.22916e+02   2.26674e+00   2.80870e-03   2.48834e-04
   3  Sigma        8.70578e+00   2.87192e+00   1.21021e-04  -3.43732e-03


In [8]:
TF1 *f2 = new TF1("f2","[0]*TMath::Power(([1]/[2]),(x/[2]))*(TMath::Exp(-([1]/[2])))/TMath::Gamma((x/[2])+1.)");
f2->SetParameters(1,m_four_lep->GetMean(),m_four_lep->GetRMS());
f2->SetParNames ("Constant","Mean_value","Sigma");
m_four_lep->Fit("f2","","",115,130);
m_four_lep->Draw();
cz->Draw();

 FCN=4.09117 FROM MIGRAD    STATUS=FAILED        991 CALLS         992 TOTAL
                     EDM=-nan    STRATEGY= 1  ERROR MATRIX UNCERTAINTY 100.0 per cent
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     8.92090e+01           nan          -nan  -4.76863e-03
   2  Mean_value   1.16693e+02           nan          -nan  -2.22077e-02
   3  Sigma        8.86537e-01           nan          -nan          -inf


Warning in <Fit>: Abnormal termination of minimization.
Error in <TGaxis::PaintAxis>: wmin (0.000000) == wmax (0.000000)


In [9]:
TF1 *f3= new TF1("f3","landau",115,130);
m_four_lep->Fit("f3","R");
m_four_lep->Draw();
cz->Draw();

 FCN=2.69436 FROM MIGRAD    STATUS=CONVERGED     141 CALLS         142 TOTAL
                     EDM=2.21604e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.45095e+01   9.81598e+00   5.09627e-03  -3.35173e-05
   2  MPV          1.21479e+02   7.55257e-01   5.09037e-04  -1.07192e-03
   3  Sigma        1.32099e+00   6.60757e-01   5.01942e-05   5.14415e-03
